In [43]:
import pandas as pd
import pyodbc
import os
import shutil
from datetime import datetime, timedelta
from dotenv import load_dotenv
import io
import json
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import google.oauth2.credentials as oauth2_creds

# Load environment variables from .env file
load_dotenv()

# Set up database connection
server = os.getenv('CR_DWH_SERVER')
username = os.getenv('CR_UN')
password = os.getenv('CR_PW')

# Setup Google Drive variables
RAW_TARGET_FILENAME = f'weekly_supervision_hours_{datetime.now().strftime("%Y-%m-%d")}.csv'
TRANSFORMED_TARGET_FILENAME = f'weekly_supervision_hours_transformed_{datetime.now().strftime("%Y-%m-%d")}.csv'
RAW_FOLDER_ID = "1gXo3C3TPH2onk2ZWz4b_yrCYLNuhcDfX"
TRANSFORMED_FOLDER_ID = "1IkGGlqCU7lYDauWFHaKrN0-ASdb6rlm7"
TRANSFORMED_ARCHIVE_ID = "1PZMr1LnckifdoanuptBAVMWsvFEMLjD5"
CLIENT_SECRET_FILE = "client_secret.json"
SCOPES = ["https://www.googleapis.com/auth/drive.file"]


In [51]:
# Base SQL query for supervision hours data (will be modified with dynamic start date)
SUPERVISION_HOURS_SQL_TEMPLATE = f"""
WITH base AS (
    SELECT
        b.BillingEntryId,  -- if available
        b.ClientContactId,
        c.ClientFullName,
        b.ProviderContactId,
        sc.ServiceCode,
        CAST(b.ServiceStartTime AS datetime) AS ServiceStartTime,
        CAST(b.ServiceEndTime   AS datetime) AS ServiceEndTime,
        COALESCE(b.ServiceLocationName, '(Unknown)') AS ServiceLocationName
        -- , DATEADD(DAY, - (DATEPART(WEEKDAY, b.ServiceEndTime) + 5) % 7, CAST(b.ServiceEndTime AS date)) AS WeekStart
    FROM [insights].[dw2].[BillingEntriesCurrent] AS b
    INNER JOIN [insights].[insights].[ServiceCode] AS sc
        ON b.ServiceCodeId = sc.ServiceCodeId
    INNER JOIN [insights].[insights].[Client] AS c
        ON b.ClientContactId = c.ClientId
    WHERE b.ServiceEndTime >= '2025-08-01' AND b.ServiceEndTime < '2025-09-01'
      AND sc.ServiceCode IN ('97155','97153','Non-billable: PM Admin','PDS | BCBA')
),
direct AS (
    SELECT
        ClientContactId, ClientFullName, ProviderContactId,
        ServiceStartTime, ServiceEndTime, ServiceLocationName
    FROM base
    WHERE ServiceCode = '97153'
),
supervision AS (
    SELECT
        ClientContactId, ClientFullName, ProviderContactId,
        ServiceStartTime, ServiceEndTime, ServiceLocationName
    FROM base
    WHERE ServiceCode IN ('97155','Non-billable: PM Admin','PDS | BCBA')
),

-- Overlap between each direct entry and any supervision entry for the same client
overlap_raw AS (
    SELECT
        d.ClientContactId,
        d.ClientFullName,
        d.ProviderContactId AS DirectProviderId,
        s.ProviderContactId AS SupervisorProviderId,
        d.ServiceLocationName AS DirectServiceLocationName,
        s.ServiceLocationName AS SupervisorServiceLocationName,
        CASE
            WHEN d.ServiceStartTime < s.ServiceEndTime
             AND d.ServiceEndTime   > s.ServiceStartTime
            THEN DATEDIFF(
                    MINUTE,
                    CASE WHEN d.ServiceStartTime > s.ServiceStartTime THEN d.ServiceStartTime ELSE s.ServiceStartTime END,
                    CASE WHEN d.ServiceEndTime   < s.ServiceEndTime   THEN d.ServiceEndTime   ELSE s.ServiceEndTime   END
                 ) / 60.0
            ELSE 0.0
        END AS OverlapHours
        -- , d.WeekStart
    FROM direct d
    INNER JOIN supervision s
        ON s.ClientContactId = d.ClientContactId
       AND d.ServiceStartTime < s.ServiceEndTime
       AND d.ServiceEndTime   > s.ServiceStartTime
),

-- Summed overlap by client, direct–supervisor pair, and locations
overlap AS (
    SELECT
        ClientContactId,
        ClientFullName,
        DirectProviderId,
        SupervisorProviderId,
        DirectServiceLocationName,
        SupervisorServiceLocationName,
        CAST(SUM(OverlapHours) AS DECIMAL(10,2)) AS OverlapHours
        -- , WeekStart
    FROM overlap_raw
    WHERE OverlapHours > 0
    GROUP BY
        ClientContactId, ClientFullName,
        DirectProviderId, SupervisorProviderId,
        DirectServiceLocationName, SupervisorServiceLocationName
        -- , WeekStart
),

-- Total direct hours per client, direct provider, and direct location
direct_totals AS (
    SELECT
        d.ClientContactId,
        d.ClientFullName,
        d.ProviderContactId AS DirectProviderId,
        d.ServiceLocationName AS DirectServiceLocationName,
        SUM(DATEDIFF(MINUTE, d.ServiceStartTime, d.ServiceEndTime)) / 60.0 AS DirectHours_Total
        -- , d.WeekStart
    FROM direct d
    GROUP BY
        d.ClientContactId, d.ClientFullName,
        d.ProviderContactId, d.ServiceLocationName
        -- , d.WeekStart
),

-- Total overlap hours per client, direct provider, and direct location (across all supervisors)
overlap_by_direct AS (
    SELECT
        ClientContactId,
        DirectProviderId,
        DirectServiceLocationName,
        SUM(OverlapHours) AS OverlapHours_Total
        -- , WeekStart
    FROM overlap
    GROUP BY
        ClientContactId, DirectProviderId, DirectServiceLocationName
        -- , WeekStart
),

-- Direct-only = total direct minus overlapped (by direct location)
direct_only AS (
    SELECT
        dt.ClientContactId,
        dt.ClientFullName,
        dt.DirectProviderId,
        dt.DirectServiceLocationName,
        CAST(dt.DirectHours_Total - COALESCE(od.OverlapHours_Total, 0.0) AS DECIMAL(10,2)) AS DirectHours_NoSupervision
        -- , dt.WeekStart
    FROM direct_totals dt
    LEFT JOIN overlap_by_direct od
      ON od.ClientContactId = dt.ClientContactId
     AND od.DirectProviderId = dt.DirectProviderId
     AND od.DirectServiceLocationName = dt.DirectServiceLocationName
     -- AND od.WeekStart = dt.WeekStart
),

-- Names for direct-only
named_direct_only AS (
    SELECT
        do.ClientContactId,
        do.ClientFullName,
        do.DirectProviderId,
        pdir.FirstName AS DirectFirstName,
        pdir.LastName  AS DirectLastName,
        CAST(NULL AS INT) AS SupervisorProviderId,
        CAST(NULL AS NVARCHAR(100)) AS SupervisorFirstName,
        CAST(NULL AS NVARCHAR(100)) AS SupervisorLastName,
        do.DirectServiceLocationName AS DirectServiceLocationName,
        CAST(NULL AS NVARCHAR(200)) AS SupervisorServiceLocationName,
        CAST(do.DirectHours_NoSupervision AS DECIMAL(10,2)) AS DirectHours,
        CAST(0.00 AS DECIMAL(10,2)) AS SupervisionHours,
        'Direct (no supervision overlap)' AS RowType
        -- , do.WeekStart
    FROM direct_only do
    LEFT JOIN [insights].[dw2].[Contacts] pdir
      ON pdir.ContactId = do.DirectProviderId
),

-- Names for overlap rows
named_overlap AS (
    SELECT
        o.ClientContactId,
        o.ClientFullName,
        o.DirectProviderId,
        pdir.FirstName AS DirectFirstName,
        pdir.LastName  AS DirectLastName,
        o.SupervisorProviderId,
        psup.FirstName AS SupervisorFirstName,
        psup.LastName  AS SupervisorLastName,
        o.DirectServiceLocationName,
        o.SupervisorServiceLocationName,
        CAST(o.OverlapHours AS DECIMAL(10,2)) AS DirectHours,
        CAST(o.OverlapHours AS DECIMAL(10,2)) AS SupervisionHours,
        'Direct overlapped with supervision' AS RowType
        -- , o.WeekStart
    FROM overlap o
    LEFT JOIN [insights].[dw2].[Contacts] pdir
      ON pdir.ContactId = o.DirectProviderId
    LEFT JOIN [insights].[dw2].[Contacts] psup
      ON psup.ContactId = o.SupervisorProviderId
)

SELECT
    x.ClientContactId,
    x.ClientFullName,
    x.DirectProviderId,
    x.DirectFirstName,
    x.DirectLastName,
    x.SupervisorProviderId,
    x.SupervisorFirstName,
    x.SupervisorLastName,
    x.DirectServiceLocationName,
    x.SupervisorServiceLocationName,
    x.DirectHours,
    x.SupervisionHours,
    x.RowType
    -- , x.WeekStart
FROM (
    SELECT * FROM named_direct_only
    UNION ALL
    SELECT * FROM named_overlap
) x
WHERE (x.DirectHours > 0 OR x.SupervisionHours > 0)
ORDER BY
    x.ClientFullName,
    x.DirectLastName, x.DirectFirstName,
    x.SupervisorLastName, x.SupervisorFirstName,
    x.DirectServiceLocationName,
    x.RowType;
"""


In [52]:
def get_oauth_service():
    creds = None
    if os.path.exists("token.json"):
        creds = oauth2_creds.Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            # Opens a local browser window for consent
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as f:
            f.write(creds.to_json())
    return build("drive", "v3", credentials=creds)

def upload_dataframe_create_only(df: pd.DataFrame, folder_id: str, filename: str, service):
    csv_bytes = df.to_csv(index=False).encode("utf-8")
    media = MediaIoBaseUpload(io.BytesIO(csv_bytes), mimetype="text/csv", resumable=True)
    metadata = {"name": filename, "parents": [folder_id]}

    created = service.files().create(
        body=metadata,
        media_body=media,
        fields="id, name, webViewLink, webContentLink",
        supportsAllDrives=True,   # works for My Drive and Shared Drives
    ).execute()
    return created

def archive_existing_files_in_folder(service, source_folder_id: str, archive_folder_id: str, exclude_filename: str = None):
    """
    Move all files from source folder to archive folder, excluding the specified filename
    """
    try:
        # List files in the source folder
        results = service.files().list(
            q=f"'{source_folder_id}' in parents and trashed=false",
            fields="files(id, name, mimeType)",
            supportsAllDrives=True
        ).execute()
        
        files_to_move = results.get('files', [])
        
        # Filter out the exclude filename if specified
        if exclude_filename:
            files_to_move = [f for f in files_to_move if f['name'] != exclude_filename]
        
        moved_files = []
        
        for file in files_to_move:
            try:
                # Move file to archive folder
                service.files().update(
                    fileId=file['id'],
                    addParents=archive_folder_id,
                    removeParents=source_folder_id,
                    supportsAllDrives=True
                ).execute()
                
                moved_files.append(file['name'])
                print(f"Moved to archive: {file['name']}")
                
            except Exception as e:
                print(f"Error moving {file['name']}: {e}")
        
        return moved_files
        
    except Exception as e:
        print(f"Error listing files in folder: {e}")
        return []


In [53]:
# Create connection string
conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE=insights;UID={username};PWD={password}'

# Set start date for testing (last 30 days)
start_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')

# Format the SQL query with the start date
sql_query = SUPERVISION_HOURS_SQL_TEMPLATE.format(start_date=start_date)

print("\n" + "="*50 + "\n")
print(f"Testing SQL query with start_date: {start_date}")

# Execute query and return dataframe
try:
    conn = pyodbc.connect(conn_str)
    df = pd.read_sql(sql_query, conn)
    conn.close()
    
    print(f"Query executed successfully! Returned {len(df)} rows.")
    
except Exception as e:
    print(f"Error executing query: {e}")




Testing SQL query with start_date: 2025-09-18


/var/folders/c5/vpx80swj4yb9ytlnrk8w2qnr0000gn/T/ipykernel_62038/1221168631.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


Query executed successfully! Returned 5945 rows.


In [55]:
df.to_csv(f'../../data/raw_pulls/august_2005_{RAW_TARGET_FILENAME}', index=False)


In [59]:
df

,ClientContactId,ClientFullName,DirectProviderId,DirectFirstName,DirectLastName,SupervisorProviderId,SupervisorFirstName,SupervisorLastName,DirectServiceLocationName,SupervisorServiceLocationName,DirectHours,SupervisionHours,RowType
0,2464119,Aalia Arif Ibrahim,2197760,Sophie,Frisina,NaN,None,None,ORGANIZATION: NC | RAL | Apex Clinic 1460 Chap...,None,0.50,0.00,Direct (no supervision overlap)
1,2464119,Aalia Arif Ibrahim,4155808,Shanell,Hickson,NaN,None,None,ORGANIZATION: NC | RAL | Apex Clinic 1460 Chap...,None,2.50,0.00,Direct (no supervision overlap)
2,2464119,Aalia Arif Ibrahim,4186777,Juliana,Jarrett Morales,NaN,None,None,ORGANIZATION: NC | RAL | Apex Clinic 1460 Chap...,None,0.25,0.00,Direct (no supervision overlap)
3,2464119,Aalia Arif Ibrahim,4129079,Dezi,Kauer,NaN,None,None,ORGANIZATION: NC | RAL | Apex Clinic 1460 Chap...,None,33.25,0.00,Direct (no supervision overlap)
4,2464119,Aalia Arif Ibrahim,4129079,Dezi,Kauer,4186777.0,Juliana,Jarrett Morales,ORGANIZATION: NC | RAL | Apex Clinic 1460 Chap...,ORGANIZATION: NC | RAL | Apex Clinic 1460 Chap...,6.75,6.75,Direct overlapped with supervision
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5940,3549322,Zyon Benton,4186740,Shrey,Patel,NaN,None,None,ORGANIZATION: NC | RAL | Durham North Clinic 4...,None,1.00,0.00,Direct (no supervision overlap)
5941,3549322,Zyon Benton,3929777,Ryan,Pherribo,NaN,None,None,ORGANIZATION: NC | RAL | Durham North Clinic 4...,None,30.42,0.00,Direct (no supervision overlap)
5942,3549322,Zyon Benton,3929777,Ryan,Pherribo,3211945.0,Emily,Laxson,ORGANIZATION: NC | RAL | Durham North Clinic 4...,ORGANIZATION: NC | RAL | Durham North Clinic 4...,2.08,2.08,Direct overlapped with supervision
5943,3549322,Zyon Benton,3901075,Lindsey,Sellars,NaN,None,None,ORGANIZATION: NC | RAL | Durham North Clinic 4...,None,5.25,0.00,Direct (no supervision overlap)


In [60]:
direct_dict = dict(zip(df['DirectProviderId'], df['DirectFirstName'] + ' ' + df['DirectLastName']))


In [64]:
transformed_df = df.groupby([
    'DirectProviderId',
    'DirectServiceLocationName']).agg({
    'DirectHours': 'sum',
    'SupervisionHours': 'sum'
    }).reset_index()

transformed_df['PctOfDirectHoursSupervised'] = round(100*(transformed_df['SupervisionHours'] / transformed_df['DirectHours']), 2)
transformed_df = transformed_df[transformed_df['DirectServiceLocationName'].str.contains('ORGANIZATION')].reset_index(drop=True)
transformed_df['Clinic'] = [val.split('ORGANIZATION: ')[1] for val in transformed_df['DirectServiceLocationName']]
transformed_df['Clinic'] = [val.split('Clinic')[0] for val in transformed_df['Clinic']]
transformed_df['Clinic'] = [val[:-1] for val in transformed_df['Clinic']]
transformed_df['Clinic'] = [val.replace(" 8528 Unive", "") for val in transformed_df['Clinic']]
transformed_df.drop(columns=['DirectServiceLocationName'], inplace=True)
transformed_df['DirectProviderName'] = transformed_df['DirectProviderId'].map(direct_dict)
transformed_df = transformed_df[['Clinic', 'DirectProviderId', 'DirectProviderName', 'DirectHours', 'SupervisionHours', 'PctOfDirectHoursSupervised']]
transformed_df = transformed_df.sort_values(by=['Clinic', 'DirectProviderName'], ascending=True)

# Archive previous week's file if it exists (LOCAL)
transformed_folder = '../../data/transformed_supervision_weekly'
archive_folder = f'{transformed_folder}/archived'

# Check if there are any existing CSV files in the main folder (excluding archived folder)
existing_files = [f for f in os.listdir(transformed_folder) 
                 if f.endswith('.csv') and f != TRANSFORMED_TARGET_FILENAME]

# Move existing files to archive folder
for file in existing_files:
    source_path = os.path.join(transformed_folder, file)
    archive_path = os.path.join(archive_folder, file)
    
    # If file already exists in archive, add timestamp to avoid conflicts
    if os.path.exists(archive_path):
        name, ext = os.path.splitext(file)
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        archive_path = os.path.join(archive_folder, f'{name}_{timestamp}{ext}')
    
    shutil.move(source_path, archive_path)
    print(f"Moved {file} to archive folder")

# Save the new file
transformed_df.to_csv(f'{transformed_folder}/august_2005_{TRANSFORMED_TARGET_FILENAME}', index=False)
print(f"Saved new file locally: {TRANSFORMED_TARGET_FILENAME}")

transformed_df


Moved august_2005_weekly_supervision_hours_transformed.csv to archive folder
Saved new file locally: weekly_supervision_hours_transformed_2025-09-25.csv


,Clinic,DirectProviderId,DirectProviderName,DirectHours,SupervisionHours,PctOfDirectHoursSupervised
163,NC | BLR | Fletcher,3177679,Alexa Pritchard,128.25,9.25,7.21
198,NC | BLR | Fletcher,3375534,Anna Brown,124.50,10.00,8.03
309,NC | BLR | Fletcher,3861399,Ashleigh Goff,98.50,8.75,8.88
510,NC | BLR | Fletcher,4186273,Camryn Yilk,114.00,9.25,8.11
161,NC | BLR | Fletcher,3168610,Chandler Corley,88.25,8.75,9.92
...,...,...,...,...,...,...
79,VA | SWV | Rocky Mount,2478710,Douglas Crawford,21.25,5.22,24.56
607,VA | SWV | Rocky Mount,4292706,Mya Meza,12.50,4.00,32.00
396,VA | SWV | Rocky Mount,4025426,Samantha Newberry,1.75,0.00,0.00
66,VA | SWV | Rocky Mount,2302059,Sarah Salmons,1.50,0.00,0.00


In [50]:
service = get_oauth_service()

# Upload raw file
raw_file_info = upload_dataframe_create_only(df, RAW_FOLDER_ID, RAW_TARGET_FILENAME, service)
print("Created:", raw_file_info["name"])

# Archive existing files in transformed folder before uploading new one (GOOGLE DRIVE)
print("\nArchiving existing files in transformed folder...")
archived_files = archive_existing_files_in_folder(
    service, 
    TRANSFORMED_FOLDER_ID, 
    TRANSFORMED_ARCHIVE_ID, 
    exclude_filename=TRANSFORMED_TARGET_FILENAME
)

if archived_files:
    print(f"Archived {len(archived_files)} files: {', '.join(archived_files)}")
else:
    print("No files to archive")

# Upload new transformed file
print(f"\nUploading new transformed file: {TRANSFORMED_TARGET_FILENAME}")
transformed_file_info = upload_dataframe_create_only(transformed_df, TRANSFORMED_FOLDER_ID, TRANSFORMED_TARGET_FILENAME, service)
print("Created:", transformed_file_info["name"])


Created: weekly_supervision_hours_2025-09-25.csv

Archiving existing files in transformed folder...
Moved to archive: weekly_supervision_hours_transformed_2025-09-23.csv
Archived 1 files: weekly_supervision_hours_transformed_2025-09-23.csv

Uploading new transformed file: weekly_supervision_hours_transformed_2025-09-25.csv
Created: weekly_supervision_hours_transformed_2025-09-25.csv
